<a href="https://www.kaggle.com/code/toptm123/translate-summarize-agent?scriptVersionId=271105341" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Agent

To better understand the basics of flows for developing AI agent, I made a simple one that performs the following streamline task:
1. Summarize a piece of text
2. Read the summary that it has written, identify keywords, and generate questions about the summary.
3. Translate the summary into another language. 

In [1]:
pip install --upgrade transformers --target=/kaggle/working/mysitepackages

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 82.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.1/566.1 kB 26.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.9/16.9 MB 71.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 806.6/806.6 kB 33.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.3/800.3 kB 35.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.8/485.8 kB 26.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 73.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
pip install -U bitsandbytes --target=/kaggle/working/mysitepackages

  Using cached numpy-2.3.4-cp311-cp311-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (62 kB)
  Using cached packaging-25.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached filelock-3.20.0-py3-none-any.whl.metadata (2.1 kB)
  Using cached typing_extensions-4.15.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached fsspec-2025.9.0-py3-none-any.whl.metadata (10 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 20.8 MB/s eta 0:00:00
Using cached numpy-2.3.4-cp311-cp311-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl (16.9 MB)
Using cached packaging-25.0-py3-none-any.whl (66 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.8/899.8 MB 758.8 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.8/954.8 kB 19.0 MB/s eta 0:00:00


In [3]:
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer
import torch
import transformers

model_id = "tiiuae/falcon-7b-instruct"

# load the tokenizer and the model
tokenizer = AutoTokenizer.from_pretrained(model_id)


model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype="auto",
    trust_remote_code=True
)

2025-10-26 20:22:53.007303: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1761510173.226188      13 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761510173.287110      13 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/281 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.48G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/117 [00:00<?, ?B/s]

In [4]:
f = open("/kaggle/input/little-star/little_star.txt","r")
input_paragraph = "".join(f.readlines())
print(input_paragraph)

The Little Star Who Couldn’t Shine

Once upon a time, in a sky full of twinkling stars, there was a little star named Spark. Unlike the other stars, Spark couldn’t shine as brightly. No matter how hard he tried, his light stayed dim and flickery.

The other stars teased him. "You’re not a real star!" they said. "Real stars shine bright!" Spark felt sad and lonely, hiding behind passing clouds so no one would see him.

One night, a little girl named Luna sat by her window, gazing up at the sky. She loved the stars, but tonight, she was feeling sad too. Her best friend had moved away, and she missed her terribly.

Just then, Spark peeked out from behind a cloud. He saw Luna’s tear-streaked face and wished with all his heart to cheer her up. He took a deep breath—and glowed as brightly as he could.

To his surprise, his light didn’t flicker this time. It shone steady and warm, forming the shape of a smiling face in the sky. Luna gasped and giggled. "That star looks like it’s smiling at me

In [5]:
generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

Device set to use cpu


In [6]:
def generate_question(keyword,text):
    
    if keyword in text:
        return generator(f"Generate two questions based on {text}")
    else:
        return "No question generated"

In [7]:
def translate(text,target_lan = "French"):
    prompt2= f"tranlate this {target_lan} {summary}. Provide the translation only, don't add extra formatting or punctuation"
    sequences = generator(
        prompt2,
        max_new_tokens=150,
        do_sample=True,
        top_k=50, #how many answers generated
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        num_beams=4
    )
    out = sequences[0]["generated_text"]
    return out.replace(prompt2, "").strip()

In [8]:
def summarize(text):
    prompt= f"Summarize this piece of text: {text} "
    sequences = generator(
        prompt,
        max_new_tokens=150,
        do_sample=True,
        top_k=50, #how many answers generated
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
    )
    out = sequences[0]["generated_text"]
    return out.replace(prompt, "").strip()

In [9]:
summary= summarize(input_paragraph)
print(summary)

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


Summary: Spark, a little star who couldn't shine as brightly as the others, comforts a lonely girl named Luna. After she moves away, he shines in the night sky to make her feel better. The other stars realize his unique light and call him the Comfort Star.


In [10]:

cond = generate_question("star",summary)
print(cond)

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


[{'generated_text': "Generate two questions based on Summary: Spark, a little star who couldn't shine as brightly as the others, comforts a lonely girl named Luna. After she moves away, he shines in the night sky to make her feel better. The other stars realize his unique light and call him the Comfort Star.\n1. How did the lonely girl Luna feel comforted by the little star, Spark? 2. How did the other stars in the night sky know about Spark's unique light?"}]


In [11]:
translation = translate(summary)
print(translation)

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


.
La petite étoile, qui ne brillait pas aussi intensément que les autres, réconfortait une fille triste nommée Luna. Après son départ, elle brillait dans le ciel pour lui faire sentir mieux. Les autres étoiles ont vu sa lumière unique et l'ont appelé le Confort Star.
